<a href="https://www.kaggle.com/code/dataminingee/resnet50-cat-dog-classifier?scriptVersionId=145052064" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Library

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

## Get and Split Data

In [ ]:
data_dir = '/kaggle/input/hw2q5zip/Dataset_OpenCvDl_Hw2_Q5/training_dataset'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
print(train_ds.class_names)

In [ ]:
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(img_width,img_height,3))
x = net.output
x = Flatten()(x)

In [ ]:
# increase DropOut layer
x = Dropout(0.5)(x)

# increase Dense layer，using softmax generate each category probability
output_layer = Dense(1, activation='sigmoid', name='sigmoid')(x)

# setting freeze layer and training layer
net_final = Model(inputs=net.input, outputs=output_layer)

# Sigmoid Focal Cross Entropy

In [ ]:
import tensorflow_addons as tfa
# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
loss_function = tfa.losses.SigmoidFocalCrossEntropy(alpha=0.4, gamma=1.0)

# loss_function = tf.keras.losses.BinaryCrossentropy()

net_final.compile(optimizer=Adam(learning_rate=8e-5),
                  loss=loss_function, metrics=['accuracy'])

In [ ]:
# train model
history = net_final.fit(train_ds,validation_data=val_ds, epochs=20)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(history.history['loss'], color='b', label='train loss')
plt.plot(history.history['val_loss'], color='r', label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], color='b', label='train acc')
plt.plot(history.history['val_accuracy'], color='r', label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# Save Model
net_final.save('Resnet50model_Focal.h5')

# Binary Cross Entropy

In [ ]:
loss_function = tf.keras.losses.BinaryCrossentropy()

net_final.compile(optimizer=Adam(learning_rate=8e-5),
                  loss=loss_function, metrics=['accuracy'])

In [ ]:
# Train Model
history = net_final.fit(train_ds,validation_data=val_ds, epochs=20)

In [ ]:
# plot the loss
plt.plot(history.history['loss'], color='b', label='train loss')
plt.plot(history.history['val_loss'], color='r', label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], color='b', label='train acc')
plt.plot(history.history['val_accuracy'], color='r', label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# Save Model
net_final.save('Resnet50model_binary.h5')

In [ ]:
focalmodel = tf.keras.models.load_model('/kaggle/working/Resnet50model_Focal.h5')
loss_focal, accuracy_focal = focalmodel.evaluate(val_ds,verbose=2)

In [ ]:
binmodel = tf.keras.models.load_model('/kaggle/working/Resnet50model_binary.h5')
loss_bin, accuracy_bin = binmodel.evaluate(val_ds,verbose=2)

In [ ]:
x = [1, 2]
accuracy_bin = accuracy_bin * 100
accuracy_focal = accuracy_focal * 100
y = [accuracy_bin, accuracy_focal]
plt.bar(x,y,tick_label=['Binary Cross Entropy', 'Focal Loss'])
plt.ylabel('Accuracy(%)')
plt.title('Accuracy Comparison')
plt.text(1, accuracy_bin, '{:.2f}'.format(round(accuracy_bin,2)))
plt.text(2, accuracy_focal, '{:.2f}'.format(round(accuracy_focal,2)))
plt.show()
plt.savefig('AccuracyComparison')